In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [2]:
import sys

linewidth = 100
formatter = {
    'float': lambda num: f'{num:5.1f}',
    'int': lambda num: f'{float(num):5.1f}',
}
print_args = {
    'threshold': sys.maxsize,
#     'threshold': 100,
    'linewidth': 100,
    'formatter': formatter,
    'edgeitems': 6,
}

In [3]:
import numpy as np
from qtm.constants import RYDBERG

from qtm.lattice import RealLattice, ReciLattice
from qtm.gspace import GSpace, GkSpace

alat = 5.107
latvec_alat = 0.5 * np.array([
    [ 1,  1,  1],
    [-1,  1,  1],
    [-1, -1,  1]
])

reallat = RealLattice.from_alat(alat, *latvec_alat)
recilat = ReciLattice.from_reallat(reallat)

ecutwfn = 40 * RYDBERG
grho = GSpace(recilat, 4 * ecutwfn)
gwfn = GkSpace(grho, [0., 0., 0.])
# print(grho.grid_shape, grho.size_g)
# print(gwfn.grid_shape, gwfn.size_g)

In [4]:
gspc = grho

arr_g = gspc.create_buffer_g(1)
arr_r = gspc.create_buffer_r(1)
print(arr_g.shape, arr_r.shape)

rng = np.random.default_rng(12345)

arr_g[:] = rng.random(gspc.size_g) \
    + 1j * rng.random(gspc.size_g)

arr_r[:] = 0
arr_r.put(gspc.idxgrid, arr_g)

arr_r = np.fft.ifftn(arr_r.reshape(gspc.grid_shape))
arr_r = arr_r.reshape((1, -1))

out = gspc.create_buffer_r(1)
gspc.g2r(arr_g, out)
print(np.allclose(out, arr_r))

out = gspc.create_buffer_g(1)
gspc.r2g(arr_r, out)
print(np.allclose(out, arr_g))

(1, 2243) (1, 5832)
True
True


In [5]:
print(np.all(np.sort(gspc.idxgrid) == gspc.idxgrid))

True


In [6]:
from qtm.mpi.comm import QTMComm
from qtm.mpi.gspace import DistGSpace
from qtm.gspace.fft.utils import cryst2idxgrid

class DummyComm(QTMComm):
    
    def __init__(self, size, rank):
        self.size, self.rank = size, rank
    
    def Get_size(self):
        return self.size
    
    def Get_rank(self):
        return self.rank

size = 4

nx, ny, nz = gspc.grid_shape
idxgrid = gspc.idxgrid
ix, iy, iz = np.unravel_index(idxgrid, gspc.grid_shape, order='C')

l_iglocal = []
for rank in range(size):
    comm = DummyComm(size, rank)
    dist_gspc = DistGSpace(comm, gspc)
    # print(dist_gspc.size_g)
    l_iglocal.append(dist_gspc.ig_loc)

# igall = np.concatenate(l_iglocal)
# print(np.all(igall[dist_gspc.gather2glob] 
#              == np.arange(gspc.size_g, dtype='i8')))


# idx = np.argsort(
#     np.searchsorted(dist_gspc.iyz_key, iy*nz + iz),
#     kind='stable'
# )
# print(np.nonzero(
#     np.arange(gspc.size_g, dtype='i8')[idx] != igall
# ))

True
True
True
True


In [7]:
import ipyparallel as ipp
print(ipp.version_info)

nproc = 4
cluster = ipp.Cluster(engines='mpi', n=nproc, shutdown_atexit=False)
print(cluster)
client = cluster.start_and_connect_sync(activate=True)
view = client[:]
client.ids

(8, 6, 1)
<Cluster(cluster_id='1690363503-3irn', profile='default')>
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.51s/engine]


[0, 1, 2, 3]

In [8]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1690363503-3irn                  True          4 MPI


In [9]:
%autopx --block --group-outputs=engines

%autopx enabled


In [10]:
import numpy as np

from qtm.lattice import RealLattice, ReciLattice
from qtm.gspace import GSpace, GkSpace

from qtm.constants import RYDBERG

alat = 5.107
latvec_alat = 0.5 * np.array([
    [ 1,  1,  1],
    [-1,  1,  1],
    [-1, -1,  1]
])
ecutwfn = 40 * RYDBERG

reallat = RealLattice.from_alat(alat, *latvec_alat)
recilat = ReciLattice.from_reallat(reallat)

grho = GSpace(recilat, 4 * ecutwfn)
gwfn = GkSpace(grho, [0., 0., 0.])

In [11]:
gspc = grho

arr_g = gspc.create_buffer_g(1)
arr_r = gspc.create_buffer_r(1)

rng = np.random.default_rng(12345)
arr_g[:] = rng.random(gspc.size_g) \
    + 1j * rng.random(gspc.size_g)

arr_r[:] = 0
arr_r.put(gspc.idxgrid, arr_g)

arr_r = np.fft.ifftn(arr_r.reshape(gspc.grid_shape))
arr_r = arr_r.reshape((1, -1))

out = gspc.create_buffer_r(1)
gspc.g2r(arr_g, out)
print(np.allclose(out, arr_r))

out = gspc.create_buffer_g(1)
gspc.r2g(arr_r, out)
print(np.allclose(out, arr_g))

[stdout:1] True
True


[stdout:2] True
True


[stdout:0] True
True


[stdout:3] True
True


In [12]:
from mpi4py.MPI import COMM_WORLD
from qtm.mpi.comm import QTMComm
from qtm.mpi.gspace import DistGSpace

comm_world = QTMComm(COMM_WORLD)
size, rank = comm_world.size, comm_world.rank
dist_gspc = DistGSpace(comm_world, gspc)

[stdout:1] True


[stdout:2] True


[stdout:3] True


[stdout:0] True


In [14]:
# from qtm.mpi.utils import scatter_range, scatter_len

# nx, ns = (5, 9)

# # Creating global data and slicing it for local data
# glob_shape = (nx, ns)
# glob_array = np.arange(np.prod(glob_shape)).reshape(glob_shape)

# loc_range = scatter_range(range(glob_shape[1]), size, rank)
# loc_start, loc_stop = loc_range.start, loc_range.stop
# loc_shape = (glob_shape[0], loc_stop - loc_start)

# loc_array = np.empty(loc_shape)
# loc_array[:] = glob_array[:, loc_start:loc_stop]
# print()
# # print(loc_array)

# inp = loc_array
# out = np.empty((glob_shape[1], scatter_len(glob_shape[0], size, rank)))

# dist_gspc._transpose(inp, out)
# # print(out)

# inp[:] = 0
# dist_gspc._transpose(out, inp)
# # print(inp)

In [15]:
# print(dist_gspc.ig_loc)

In [16]:
from qtm.mpi.utils import scatter_len, scatter_range

arr_r_, arr_g_ = arr_r[0].reshape(gspc.grid_shape), arr_g[0]

ix_loc = scatter_range(range(dist_gspc.nz), size, rank)
ix_start, ix_stop = ix_loc.start, ix_loc.stop
arr_r_loc = arr_r_[..., ix_start:ix_stop]
arr_g_loc = arr_g_[dist_gspc.ig_loc]

print(arr_r_loc.shape, arr_g_loc.shape)

[stdout:0] (18, 18, 5) (667,)


[stdout:3] (18, 18, 4) (624,)


[stdout:2] (18, 18, 4) (441,)


[stdout:1] (18, 18, 5) (511,)


In [17]:
out = dist_gspc.g2r(arr_g_loc)
print(out.shape, arr_r_loc.ravel().shape)
print(np.allclose(out , arr_r_loc.ravel()))

[stdout:1] (1620,) (1620,)
True


[stdout:2] (1296,) (1296,)
True


[stdout:3] (1296,) (1296,)
True


[stdout:0] (1620,) (1620,)
True


In [18]:
out = dist_gspc.r2g(arr_r_loc.ravel())
print(np.allclose(out , arr_g_loc.ravel()))
print(out.shape)

[stdout:0] True
(667,)


[stdout:2] True
(441,)


[stdout:1] True
(511,)


[stdout:3] True
(624,)


In [19]:
%autopx --block --group-outputs=engines

%autopx disabled


In [20]:
cluster.stop_cluster_sync()

In [21]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1690365451-v2ci                  False         4 MPI
